# Applying BBVI for a simple Gaussian Model

<img src="BBVI_exercise.png">

In [13]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [14]:
# Generate data from a simple model: Normal(10, 1)
data = np.random.normal(loc = 10, scale = 1, size = 100)

# Helper function: ELBO

Calculate the exact value of the ELBO. Generally one would have to estimate this using sampling, but for this simple model we can evaluate it exactly 

In [15]:
def calculate_lower_bound(tau, q_mu):
    """
    Helper routine: Calculate ELBO. Data is the sampled x-values, anything without a star relates to the prior,
    everything _with_ a star relates to the variational posterior.
    Note that we have no nu without a star; I am simplifying by forcing this to be zero a priori

    Note: This function obviously only works when the model is as in this code challenge,
    and is not a general solution.

    :param data: The sampled data
    :param tau: prior precision for mu, the mean for the data generation
    :param alpha: prior shape of dist for gamma, the precision  of the data generation
    :param beta: prior rate of dist for gamma, the precision  of the data generation
    :param nu_star: VB posterior mean for the distribution of mu - the mean of the data generation
    :param tau_star: VB posterior precision for the distribution of mu - the mean of the data generation
    :param alpha_star: VB posterior shape of dist for gamma, the precision  of the data generation
    :param beta_star: VB posterior shape of dist for gamma, the precision  of the data generation
    :return: the ELBO
    """

    # We calculate ELBO as E_q log p(x,mu) - E_q log q(mu)
    # log p(x,z) here is log p(mu) + \sum_i log p(x_i | mu, 1)

    # E_q log p(mu)
    log_p = -.5 * np.log(2 * np.pi) - .5 * (1/tau) * (1 + q_mu**2)


    # E_q log p(x_i|mu, 1)
    for xi in data:
        log_p += -.5 * np.log(2 * np.pi) - .5 * (xi * xi - 2 * xi * q_mu + 1 + q_mu**2)

    # Entropy of mu (Gaussian)
    entropy = .5 * np.log(2 * np.pi * np.exp(1))

    return log_p + entropy

# Manual estimation of the gradient of the ELBO for the above model

In [4]:
# Gradient estimator using sampling -- vanilla BBVI
# We here assume the model X ~ Normal(mu, 1)
# with unknown mu, that in itself is Normal, mean 0 and standard deviation 1000, 
# so effectively an uniformed prior. 
# The variational dstribution for mu is also Normal, with parameter q_mu_lambda
# -- taking the role of lambda in the calculations -- and variance 1.
#
# Note:
# We can sample from a normal using:
# * np.random.normal(loc=mu, scale=1, size=1)
# We can evaluate the the normal density using
# * norm.logpdf(sample, loc = mu, scale = std. dev.)

def grad_estimate(q_mu_lambda, samples = 1):
    # sum_grad_estimate will hold the sum as we move along over the <samples> samples. 
    sum_grad_estimate = 0
    for i in range(samples):
        # Sample one example from current best guess for the variational distribution
        mu_sample = np.random.normal(loc=q_mu_lambda, scale=1, size=1)
        
        # Now we want to calculate the contribution from this sample, namely 
        # [log p(x, mu_sample) - log q(mu|lambda) ] * grad( log q(mu_sample|lambda) )
        #
        value = ?

        # Next grad (log q(mu_sample|lambda))
        # The Normal distribution gives the score function with known variance as <value> - <mean>
        grad_q = ?

        
        # grad ELBO for this sample is therefore in total given by
        sum_grad_estimate = sum_grad_estimate + grad_q * value
       
    # Divide by number of samples to get average value -- the estimated expectation  
    return sum_grad_estimate/samples

# Perform BBVI using the estimated gradient

In [11]:
import time
no_loops = 500
sample_count = 100
##### Starting point
q_mu = -10
start = time.time()
elbos = []
lr = 1E-4 


#loop a couple of times
for t in range(no_loops):
    elbos.append(calculate_lower_bound(1000, q_mu))
    q_grad = grad_estimate(q_mu, samples=sample_count)
    q_mu = q_mu + lr * q_grad

print("{:4d} sample(s) -- Estimate: {:9.5f}; error {:5.1f}%  --  Calc.time: {:5.2f} sec.".format(
    sample_count, float(q_mu), float(10*np.abs(q_mu-10)), time.time() - start))

 100 sample(s) -- Estimate:   9.91943; error   0.8%  --  Calc.time: 17.40 sec.


### Exercise
* Try varying the number of samples used for estimating the gradient. What effect does it have on the results?